Displays some common plots given paths to nonreference and reference OceanView USB2000+ Ocean Optics spectrometer data. See code comments for usage.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

#  Make the notebook display matplotlib graphs
%matplotlib widget

path_to_data_folder = "C:/Users/jrsha/UIUC/Abbamonte_research/Data" # folder where data is stored

### Given a file path to a tab-separated text file outputted by OceanView, reads the data into a dataframe 
###     renames columns to be more descriptive 
def read_ocean_optics(file_path):
    df = pd.read_csv(file_path, sep='\t')
    df.rename( columns={'Unnamed: 1':'Intensity'}, inplace=True )
    df.rename( columns={'QEP04302_1:101':'Wavelength (nm)'}, inplace=True )
    df.drop('Unnamed: 2', axis=1, inplace=True) # empty column remnant of how data is read

    # for xray room computer
    # df = pd.read_csv(file_path, header=None, sep='\t')
    # df.rename( columns={1:'Intensity'}, inplace=True )
    # df.rename( columns={0:'USB2+F01863_1:101 Wavelength (nm)'}, inplace=True )
    return df

### Displays plots that are common when given reference and nonreference data. 
### Parameters are path_to_nonreference_data followed by path_to_reference_data followed by experiment title.
def display_common_plots_for_reference_and_nonreference_data(path_to_nonreference_data, path_to_reference_data, title):
    nonreference_df = read_ocean_optics(path_to_nonreference_data)
    reference_df = read_ocean_optics(path_to_reference_data)

    nonreference_df.plot(x='Wavelength (nm)', kind='line', title=(title + " Nonreference"))
    plt.ylabel("Intensity")

    reference_df.plot(x='Wavelength (nm)', kind='line', title=(title + " Reference"))
    plt.ylabel("Intensity")

    display_plot_of_scaled_data_fitted_to_reference(nonreference_df, reference_df, title)


### Displays plots that are common when given reference and nonreference data. 
### Parameters are nonreference_df followed by reference_df followed by experiment title.
def display_plot_of_scaled_data_fitted_to_reference(nonreference_df, reference_df, title):
    def scale_func(x, a):
        return a * x
    

    # fit data ignoring saturated values
    filter = 415
    popt, _ = curve_fit(scale_func, nonreference_df[nonreference_df['Wavelength (nm)'] > filter]['Intensity'], reference_df[reference_df['Wavelength (nm)'] > filter]['Intensity'])

    # popt, _ = curve_fit(scale_func, nonreference_df['Intensity'], reference_df['Intensity'])

    scaled_data_fitted_to_reference_df = reference_df.copy()
    scaled_data_fitted_to_reference_df.rename( columns={'Intensity': 'Reference'}, inplace=True )
    scaled_data_fitted_to_reference_df['Nonreference'] = nonreference_df['Intensity']
    scaled_data_fitted_to_reference_df['Scaled Nonreference'] = nonreference_df['Intensity'] * popt[0]
    

    scaled_data_fitted_to_reference_df.plot(x='Wavelength (nm)', kind='line', title=(title + " Nonreference Scaled to Reference"))
    
    diff = reference_df.copy()
    diff['Intensity'] = scaled_data_fitted_to_reference_df['Nonreference'] - scaled_data_fitted_to_reference_df['Reference']
    diff.plot(x='Wavelength (nm)', kind='line', title=(title + " Difference Nonreference Scaled to Reference"))
    
    # plt.xlim(594, 630)
# display common plots usage
# display_common_plots_for_reference_and_nonreference_data((path_to_data_folder + "/12.17/ex2_stilbene_uv_center.txt"), 
#                                                          (path_to_data_folder + "/12.17/ex2_stilbene_nouv1.txt"), 
#                                                          "UV and No UV Stilbene")

stilbene_uv_df = read_ocean_optics(path_to_data_folder + "/12.17/ex2_stilbene_uv_center.txt")
stilbene_nouv_df = read_ocean_optics(path_to_data_folder + "/12.17/ex2_stilbene_nouv1.txt")
kimtech_uv_df = read_ocean_optics(path_to_data_folder + "/12.17/ex2_kimtech_uv_center.txt")
kimtech_nouv_df = read_ocean_optics(path_to_data_folder + "/12.17/ex2_kimtech_nouv.txt")


nonreference_df = stilbene_uv_df - stilbene_nouv_df 
reference_df = kimtech_uv_df - kimtech_nouv_df

display_plot_of_scaled_data_fitted_to_reference(nonreference_df, reference_df, "Stilbene and Kimtech with no UV subtracted")

ValueError: `ydata` must not be empty!